In [ ]:
! mkdir -p datasets
%cd datasets
! wget -nc https://raw.githubusercontent.com/pablonoya/zigzag-ml/master/datasets/Iris_mod.csv
! wget -nc https://raw.githubusercontent.com/pablonoya/zigzag-ml/master/datasets/housing.csv
%cd ..

# Preprocesamiento de datos
En el mundo real, los datos que obtengamos pueden ser erróneos porque estarán incompletos, duplicados o no se podrán usar tal cual, porque los algoritmos de *machine learning* usan **sólo números** 🔢.  

Veremos cómo afrontar este tipo de situaciones con un nuevo dataset, el [Iris Dataset](https://www.kaggle.com/uciml/iris) de kaggle.

In [ ]:
import pandas as pd

data_iris = pd.read_csv('./datasets/Iris_mod.csv')
data_iris.head()

Como puedes ver, tenemos un dataset con las medidas de largo y ancho del sépalo y el pétalo de una flor de iris, cuya especie se menciona en la última columna.

También tenemos la columna "Id" pero no es necesaria, así que vamos a eliminarla con el método `drop`

In [ ]:
data_iris.drop(columns=['Id'], inplace=True)

¿Cuántas especies tenemos? usemos la función `nunique` para contar los valores únicos de la columna "Species"

In [ ]:
data_iris['Species'].nunique()

Esto **ignora** los valores **nulos**, veamos cuántos tenemos en total.

In [ ]:
data_iris.isna().sum()

# Solucionando valores nulos
Tenemos exactamente tres valores nulos en cada columna. ¿Concidencia? ~~no lo creo~~.  
En realidad modifiqué el dataset para propósitos educativos 😄 y añadí una fila de nulos que podemos verla usando `tail`

In [ ]:
data_iris.tail(2)

Estos aparecen como **NaN** siglas de *Not a Number* o **No es un Número**, para tratarlos podemos **eliminar las filas o columnas** que contengan algún valor nulo o no permitido 😵.  
Otra opción es **reemplazarlos** con la **media** o la **moda** de toda su columna 🤔.

Antes de intentar cualquier solución crearemos una **copia** de nuestro data_iris usando `copy` para preservar el original, si no usas copy sólo tendrás una referencia al objeto original, como pasa con las listas en Python 🐍.

In [ ]:
df = data_iris.copy()

## Eliminar filas
Es recomendable cuando tenemos **muchos datos y pocos son nulos**, pues no perdemos demasiada información.

Usaremos el método `dropna` que recibe el parámetro `axis` o eje, que corresponde con la dimensión, podemos utilizar`0` o `"index"` para el caso de las **filas**.
La función eliminará cualquier fila que contenga algún valor nulo 😵.

In [ ]:
df = df.dropna(axis=0)
df.tail()

## Eliminar columnas
La usamos **si la columna no es importante**, o si **tenemos demasiados valores nulos**, pues no podríamos recuperar información.

Sólo cambiamos`axis` a `1` o `'columns'` para el caso de las **columnas**.

In [ ]:
df = data_iris.copy()

df = df.dropna('columns')
df.tail()

$\cdots$  
Recuerda siempre que **un sólo nulo basta**  para eliminar **toda** la fila o columna 😉

## Reemplazar con la media
La media es el promedio de la columna, por supuesto sólo funciona con los valores numéricos 🔢.  
Debido a esto, excluímos la columna **Species** usando `iloc`.

In [ ]:
# todas las filas, de la primera a la penúltima columna
df = data_iris.iloc[:, :-1]
df.tail(3)

sklearn tiene el objeto `SimpleImputer` para reemplazar valores, reemplazará los nulos por defecto.  
Sólo debemos indicarle la `strategy` o estrategia para reemplazar los valores, que será `"mean"` en este caso. 

In [ ]:
from sklearn.impute import SimpleImputer

df = data_iris.iloc[:, :-1]
imp = SimpleImputer(strategy="mean")

df_inputed_mean = imp.fit_transform(df)

# veamos la última fila
df_inputed_mean[-1]

Podemos comprobar que sean los valores correctos usando el método `mean` sobre el *DataFrame*

In [ ]:
df.mean()

## Reemplazar con la moda
La moda es el valor que más se repite, por lo que podemos usar texto 🔡.  
Usaremos también `SimpleImputer` cambiando la `strategy` a `"most_frequent"`

In [ ]:
df = data_iris.copy()

imp = SimpleImputer(strategy="most_frequent")

df_inputed_mode = imp.fit_transform(df)
df_inputed_mode[-1]

Y podemos comprobarlo con el método `mode`

In [ ]:
df.mode()

Pandas calculó que los nulos son valores que se repiten, pero `SimpleImputer` tomó los primeros valores no nulos.

# Valores no numéricos
Para tratarlos **es necesario codificarlos**, una de las primeras ideas es asignar valores en serie como 1, 2, 3,... a cada especie, pero esto **implica que hay un orden establecido** entre especies.
El modelo podría entender algo como 3 > 2 > 1, aprendiendo que cierta especie es mejor o peor que otra 😅.

En su lugar debemos indicar que son valores de diferente **categoría** con un código que indique algo como
> Es Iris setosa y no otra cosa

| # | Iris setosa | Otra cosa |
|---|-------------|-----------|
| 1 |   sí   |   no   |
| 2 |   sí   |   no   |
| 3 |   no   |   si   |

Tendremos **una columna por categoría**, marcando 1 a la que pertenece y 0 a las demás, esto se conoce como **One-hot encoding** y podemos obtenerla con el método `get_dummies` de pandas 🐼

In [ ]:
df_one_hot = pd.get_dummies(df)
df_one_hot.head()

Por defecto actuará con todos los valores con el *dtype* `object`, pero también puedes tener un dataset con **categorías en forma de números**, como decíamos antes 1, 2, 3, ...  
Asegúrate de informarte sobre las columnas del dataset, podrías encontrar esta información en la [fuente](https://www.kaggle.com/uciml/iris) de cada dataset 😉.

Puedes verificar los `dtypes` de cada columna con ese atributo:

In [ ]:
df.dtypes

También podemos usar `OneHotEncoder` de sklearn, pero sólo enviaremos las columnas que queramos transformar y no admite que los datos tengan valores nulos.  
Por defecto retorna una [matriz dispersa](https://es.wikipedia.org/wiki/Matriz_dispersa) pero podemos cambiarlo cambiando el parámetro `sparse` a falso.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# sólo la última columna
df_species = df_inputed_mode[:, [-1]]

one_hot = OneHotEncoder(sparse=False)
df_one_hot = one_hot.fit_transform(df_species)
df_one_hot[:5]

Si tuviéramos sólo dos categorías **no es necesario agregar dos columnas**, los valores 0 y 1 ya indican la **presencia o ausencia** de la feature 😄.

# Ejercicios
¿Qué otras estrategias pueden ser útiles? prueba las que tiene `SimpleImputer` de sklearn 

In [ ]:
# revisa la documentación


Utiliza alguna estrategia con el dataset de casas, los datos nulos estaban en "total_bedrooms", evalúa qué tanto mejora el modelo utilizando sólo esa feature y la variable objetivo "median_house_value"

In [ ]:
# total_bedrooms es una cantidad


Con todo lo aprendido ya vamos dominando las bases del machine learning 😉.  
Es más, ahora la máquina puede aprender a **distinguir categorías o clases** 🤯.

> Ten las medídas de sépalo, pétalo y dime ¿es setosa u otra cosa?  

$\cdots$  
Responderá, cuando aprenda [clasificación](8_Clasificacion.ipynb).